In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [44.8 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,400 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,274 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ub

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://ale-work-123.s3.us-west-1.amazonaws.com/heart_statlog_cleveland_hungary_final_clean.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("heart_statlog_cleveland_hungary_final_clean.csv"), sep=",", header=True)

df.show()

+---+---+---------------+------------+-----------+-------------------+-----------+--------------+---------------+-------+--------+------+
|age|sex|chest_pain_type|resting_bp_s|cholesterol|fasting_blood_sugar|resting_ecg|max_heart_rate|exercise_angina|oldpeak|ST_slope|target|
+---+---+---------------+------------+-----------+-------------------+-----------+--------------+---------------+-------+--------+------+
| 40|  1|              2|         140|        289|                  0|          0|           172|              0|    0.0|       1|     0|
| 49|  0|              3|         160|        180|                  0|          0|           156|              0|    1.0|       2|     1|
| 37|  1|              2|         130|        283|                  0|          1|            98|              0|    0.0|       1|     0|
| 48|  0|              4|         138|        214|                  0|          0|           108|              1|    1.5|       2|     1|
| 54|  1|              3|         

In [5]:
df.createOrReplaceTempView('heart_disease')

In [38]:
## Type of chest pain compared to their cholesterol levels and if they have heart disease or not
q1 = """
    select chest_pain_type as chest_pain_type, cholesterol as cholesterol, target as heart_disease
    from heart_disease
    where cholesterol != 0
    order by chest_pain_type
"""

spark.sql(q1).show()

+---------------+-----------+-------------+
|chest_pain_type|cholesterol|heart_disease|
+---------------+-----------+-------------+
|              1|        291|            1|
|              1|        273|            1|
|              1|        160|            0|
|              1|        270|            1|
|              1|        252|            0|
|              1|        223|            0|
|              1|        258|            1|
|              1|        272|            1|
|              1|        200|            1|
|              1|        308|            0|
|              1|        181|            0|
|              1|        237|            0|
|              1|        216|            1|
|              1|        171|            0|
|              1|        139|            0|
|              1|        156|            1|
|              1|        234|            1|
|              1|        249|            0|
|              1|        211|            0|
|              1|        199|   

In [ ]:
q2 = """
    select chest_pain_type as chest_pain_type, resting_ecg as resting_ecg, target as heart_disease
    from heart_disease
    order by heart_disease
"""
spark.sql(q2).show()

+---------------+-----------+-------------+
|chest_pain_type|resting_ecg|heart_disease|
+---------------+-----------+-------------+
|              2|          0|            0|
|              3|          0|            0|
|              2|          0|            0|
|              4|          1|            0|
|              2|          0|            0|
|              3|          0|            0|
|              2|          0|            0|
|              2|          0|            0|
|              2|          0|            0|
|              2|          0|            0|
|              3|          0|            0|
|              2|          0|            0|
|              2|          0|            0|
|              2|          0|            0|
|              2|          0|            0|
|              1|          0|            0|
|              2|          1|            0|
|              2|          1|            0|
|              2|          1|            0|
|              3|          0|   

In [ ]:
q3 = """
    select age as age, round(avg(resting_bp_s), 2) as avg_resting_bp_s, round(avg(cholesterol), 2) as cholesterol
    from heart_disease
    group by age
    order by age
"""
spark.sql(q3).show()

+---+----------------+-----------+
|age|avg_resting_bp_s|cholesterol|
+---+----------------+-----------+
| 28|           130.0|      132.0|
| 29|           130.0|     236.67|
| 30|           170.0|      237.0|
| 31|           110.0|      244.5|
| 32|           110.6|      241.2|
| 33|           110.0|      272.0|
| 34|          124.14|     163.29|
| 35|          126.27|     197.45|
| 36|          123.67|     190.33|
| 37|          127.09|     233.73|
| 38|           120.0|      146.5|
| 39|          127.33|      239.2|
| 40|          128.31|     229.38|
| 41|          120.46|      216.5|
| 42|          129.17|     212.22|
| 43|          126.67|     202.33|
| 44|          125.95|     249.58|
| 45|          129.22|     229.33|
| 46|          126.38|     217.17|
| 47|          132.53|     193.42|
+---+----------------+-----------+
only showing top 20 rows



In [ ]:
q4 = """
    select age as age, sex as sex, round(avg(resting_bp_s), 2) as avg_resting_bp_s, round(avg(cholesterol), 2) as cholesterol
    from heart_disease
    group by age, sex
    order by age
"""
spark.sql(q4).show()

+---+---+----------------+-----------+
|age|sex|avg_resting_bp_s|cholesterol|
+---+---+----------------+-----------+
| 28|  1|           130.0|      132.0|
| 29|  1|           130.0|     236.67|
| 30|  0|           170.0|      237.0|
| 31|  1|           120.0|      270.0|
| 31|  0|           100.0|      219.0|
| 32|  0|           105.0|      198.0|
| 32|  1|           112.0|      252.0|
| 33|  1|           120.0|      298.0|
| 33|  0|           100.0|      246.0|
| 34|  0|           124.0|      185.5|
| 34|  1|           124.2|      154.4|
| 35|  0|          132.67|      170.0|
| 35|  1|          123.88|     207.75|
| 36|  1|          123.67|     190.33|
| 37|  0|           125.0|     214.75|
| 37|  1|          128.29|     244.57|
| 38|  1|          121.92|     159.15|
| 38|  0|          111.67|      91.67|
| 39|  0|           114.0|     200.33|
| 39|  1|          130.67|     248.92|
+---+---+----------------+-----------+
only showing top 20 rows



In [ ]:
q5 = """
    select cholesterol as cholesterol, target as heart_disease
    from heart_disease
    where cholesterol >= 200
"""
spark.sql(q5).show()


+-----------+-------------+
|cholesterol|heart_disease|
+-----------+-------------+
|        289|            0|
|        283|            0|
|        214|            1|
|        339|            0|
|        237|            0|
|        208|            0|
|        207|            1|
|        284|            0|
|        211|            0|
|        204|            0|
|        234|            1|
|        211|            0|
|        273|            0|
|        201|            0|
|        248|            1|
|        267|            1|
|        223|            0|
|        201|            0|
|        288|            1|
|        215|            0|
+-----------+-------------+
only showing top 20 rows



In [16]:
q6 = """
    select sex as sex, round(avg(cholesterol), 2) as avg_cholesterol, round(avg(resting_bp_s), 2) as resting_bp_s, count(target) as with_heart_disease
    from heart_disease
    where cholesterol != 0 or resting_bp_s = 0
    group by sex
"""
spark.sql(q6).show()

+---+---------------+------------+------------------+
|sex|avg_cholesterol|resting_bp_s|with_heart_disease|
+---+---------------+------------+------------------+
|  0|         255.77|      131.98|               182|
|  1|         240.61|      133.12|               565|
+---+---------------+------------+------------------+



In [8]:
## Average Cholesterol and Average Resting blood pressure based on those with or without heart disease
q7 = """
    select target as heart_disease, round(avg(cholesterol), 2) as avg_cholesterol, round(avg(resting_bp_s), 2) as resting_bp_s
    from heart_disease
    where cholesterol != 0 or resting_bp_s = 0
    group by target
"""
spark.sql(q7).show()

+-------------+---------------+------------+
|heart_disease|avg_cholesterol|resting_bp_s|
+-------------+---------------+------------+
|            0|         238.77|      130.16|
|            1|         250.36|      135.77|
+-------------+---------------+------------+



In [ ]:
q8 = """
    select target as heart_disease, round(avg(chest_pain_type), 2) as avg_chest_pain_type
    from heart_disease
    group by target
"""
spark.sql(q8).show()

+-------------+-------------------+
|heart_disease|avg_chest_pain_type|
+-------------+-------------------+
|            0|               2.76|
|            1|               3.65|
+-------------+-------------------+



In [ ]:
q9 = """
    select target as heart_disease, select

In [13]:
## People with or without exercise angina and their average cholesterol
q10 = """
    select exercise_angina, count(exercise_angina), round(avg(cholesterol), 2) as avg_choleseterol
    from heart_disease
    where cholesterol != 0
    group by exercise_angina
"""
spark.sql(q10).show()

+---------------+----------------------+----------------+
|exercise_angina|count(exercise_angina)|avg_choleseterol|
+---------------+----------------------+----------------+
|              0|                   459|          240.58|
|              1|                   287|          251.12|
+---------------+----------------------+----------------+



In [26]:
## Count the number of individuals with heart disease based on their sex
q11 = """
    select sex as sex, count(target) as heart_disease
    from heart_disease
    where target == 1
    group by sex
"""
spark.sql(q11).show()

+---+-------------+
|sex|heart_disease|
+---+-------------+
|  0|           50|
|  1|          458|
+---+-------------+



In [27]:
## Count the number of individuals without heart disease based on their sex
q12 = """
    select sex as sex, count(target) as heart_disease
    from heart_disease
    where target == 0
    group by sex
"""
spark.sql(q12).show()

+---+-------------+
|sex|heart_disease|
+---+-------------+
|  0|          143|
|  1|          267|
+---+-------------+



In [ ]:
## Blood Pressure and Cholesterol based on chest pain type
q13 = """


In [ ]:
## Individuals marked with high cholesterol and heart disease
q14 = """
    select age,
    from heart_disease
    where cholesterol >= 240
    group by age

In [32]:
q14 = """
    select age, round(avg(cholesterol), 2) as cholesterol, round(avg(chest_pain_type))
    from heart_disease
    where cholesterol >= 240
    group by age
    order by age ASC
"""
spark.sql(q14).show()

+---+-----------+
|age|cholesterol|
+---+-----------+
| 29|      253.0|
| 31|      270.0|
| 32|      391.5|
| 33|      272.0|
| 35|     277.75|
| 36|      303.5|
| 37|      269.6|
| 38|      287.0|
| 39|      286.0|
| 40|     323.83|
| 41|     277.18|
| 42|     278.13|
| 43|     277.09|
| 44|     325.14|
| 45|     288.83|
| 46|      273.0|
| 47|      261.2|
| 48|     276.13|
| 49|     280.29|
| 50|     288.88|
+---+-----------+
only showing top 20 rows



In [36]:
cp1 = """
    select chest_pain_type, cholesterol, resting_bp_s, exercise_angina, target
    from heart_disease
    where chest_pain_type = 1 and cholesterol != 0 and resting_bp_s != 0
    order by target
"""
spark.sql(cp1).show()

+---------------+-----------+------------+---------------+------+
|chest_pain_type|cholesterol|resting_bp_s|exercise_angina|target|
+---------------+-----------+------------+---------------+------+
|              1|        223|         100|              0|     0|
|              1|        160|         120|              0|     0|
|              1|        193|         160|              0|     0|
|              1|        308|         130|              0|     0|
|              1|        249|         110|              0|     0|
|              1|        237|         170|              0|     0|
|              1|        171|         120|              0|     0|
|              1|        252|         140|              0|     0|
|              1|        181|         139|              0|     0|
|              1|        139|         135|              0|     0|
|              1|        211|         110|              1|     0|
|              1|        199|         140|              1|     0|
|         

In [37]:
cp2 = """
    select chest_pain_type, cholesterol, resting_bp_s, exercise_angina, target
    from heart_disease
    where chest_pain_type = 2 and cholesterol != 0 and resting_bp_s != 0
    order by target
"""
spark.sql(cp2).show()

+---------------+-----------+------------+---------------+------+
|chest_pain_type|cholesterol|resting_bp_s|exercise_angina|target|
+---------------+-----------+------------+---------------+------+
|              2|        264|         150|              0|     0|
|              2|        241|         190|              0|     0|
|              2|        216|         140|              0|     0|
|              2|        283|         130|              0|     0|
|              2|        224|         140|              0|     0|
|              2|        208|         110|              0|     0|
|              2|        245|         130|              0|     0|
|              2|        194|         160|              0|     0|
|              2|        214|         150|              0|     0|
|              2|        253|         100|              0|     0|
|              2|        201|         120|              0|     0|
|              2|        202|         110|              0|     0|
|         

In [ ]:
cp3 = """
    select chest_pain_type, cholesterol, resting_bp_s, exercise_angina, target
    from heart_disease
    where chest_pain_type = 1 and cholesterol != 0 and resting_bp_s != 0
    order by target
"""
spark.sql(cp1).show()

In [ ]:
cp4 = """
    select chest_pain_type, cholesterol, resting_bp_s, exercise_angina, target
    from heart_disease
    where chest_pain_type = 1 and cholesterol != 0 and resting_bp_s != 0
    order by target
"""
spark.sql(cp1).show()